In [12]:
import polars as pl

In [13]:
data = pl.read_csv("~/PycharmProjects/explanaible-steam-nlp/data/dataset.csv")

In [14]:
data.schema

Schema([('app_id', Int64),
        ('app_name', String),
        ('review_text', String),
        ('review_score', Int64),
        ('review_votes', Int64)])

In [18]:
data.describe()

statistic,app_id,app_name,review_text,review_score,review_votes
str,f64,str,str,f64,f64
"""count""",6.417106e6,"""6233872""","""6409801""",6.417106e6,6.417106e6
"""null_count""",0.0,"""183234""","""7305""",0.0,0.0
"""mean""",227469.499959,null,null,0.639499,0.147245
"""std""",126045.06578,null,null,0.768792,0.35435
"""min""",10.0,""" 神明的一天世界-God's One Day World""","""I implore you to get a little…",-1.0,0.0
"""25%""",201810.0,null,null,1.0,0.0
"""50%""",239160.0,null,null,1.0,0.0
"""75%""",305620.0,null,null,1.0,0.0
"""max""",565340.0,"""拯救大魔王2 Rescue the Great Demon …","""󠀁󠀁󠀁󠀁󠀁""",1.0,1.0


# Eliminar Nulos
Eliminamos los nulos debido a que son las colummnas que contienen texto, es decir, no se pueden imputar de una manera sencilla para este analísis

In [19]:
data = data.drop_nulls()
data.describe()

statistic,app_id,app_name,review_text,review_score,review_votes
str,f64,str,str,f64,f64
"""count""",6.226728e6,"""6226728""","""6226728""",6.226728e6,6.226728e6
"""null_count""",0.0,"""0""","""0""",0.0,0.0
"""mean""",226694.093372,null,null,0.646493,0.14673
"""std""",125466.779858,null,null,0.76292,0.353837
"""min""",10.0,""" 神明的一天世界-God's One Day World""","""I implore you to get a little…",-1.0,0.0
"""25%""",201810.0,null,null,1.0,0.0
"""50%""",239140.0,null,null,1.0,0.0
"""75%""",304460.0,null,null,1.0,0.0
"""max""",563430.0,"""拯救大魔王2 Rescue the Great Demon …","""󠀁󠀁󠀁󠀁󠀁""",1.0,1.0


In [ ]:
import langid
# To-do
# procesar por chunks
+
def detectar_idioma(texto):
    if texto is None or texto.strip() == "":
        return None
    idioma, _ = langid.classify(texto)
    return idioma

# Aplicar detección con Polars
data = data.with_columns(
    pl.col("review_text").map_elements(detectar_idioma, return_dtype=pl.Utf8).alias("idioma")
)

# Filtrar solo los textos en inglés
df = data.filter(pl.col("idioma") == "en")

# (opcional) eliminar la columna auxiliar
df = df.drop("idioma")

df.head()